In [117]:
import pandas as pd
url = "https://raw.githubusercontent.com/jwjwyoung/RecentUpgradeFailureStudy/main/recent_upgrade_failures.csv"
df = pd.read_csv(url)

### generate Table 1

In [ ]:
issues = list(df['Issue key'])
app = ['cassandra', 'hbase', 'hdfs','kafka', 'mapreduce','mesos', 'yarn', 'zookeeper']
dic = {a: 0 for a in app}
for a in app:
    n = len([i for i in issues if a in i.lower()])
    dic[a] = n
table1 = pd.DataFrame([list(dic), dic.values()])
print(table1)

### Finding 1: Upgrade failures have significantly higher priority than regular failures.

In [89]:

# bugs not cassandra
pd.options.display.float_format = '{:,.1f} %'.format
ca_issues = df[df['Issue key'].str.contains("CASSANDRA")]
non_ca_issues = df[~df['Issue key'].str.contains("CASSANDRA")]
ca_perc = ca_issues.groupby('priority')['Issue key'].count().apply(lambda x:100 * x /len(ca_issues)).to_frame()
non_ca_perc = non_ca_issues.groupby('priority')['Issue key'].count().apply(lambda x:100 * x / (len(non_ca_issues))).to_frame()

In [90]:
non_ca_perc

,Issue key
priority,
Blocker,38.0 %
Critical,15.2 %
Major,44.3 %
Minor,2.5 %


In [91]:
ca_perc

,Issue key
priority,
Low,6.8 %
Normal,61.4 %
Urgent,20.5 %
normal,11.4 %


### generate Table 2

In [144]:
keys = ['rolling', 'data', 'performance', 'part', 'incorrect', 'unknown']
cs = []
ccs = []
cps = []
whole_df = df.loc[~((df.symptom.str.lower().str.contains(keys[0])) 
               | df.symptom.str.lower().str.contains(keys[1])
               | df.symptom.str.lower().str.contains(keys[2])
               | df.symptom.str.lower().str.contains(keys[3]) 
               | df.symptom.str.lower().str.contains(keys[4]) 
               | df.symptom.str.lower().str.contains(keys[5]))]
severity_dfs = [whole_df]
for k in keys:
    filtered_df = df[df['symptom'].str.lower().str.contains(k)]
    severity_dfs.append(filtered_df)
    
for filtered_df in severity_dfs:
    filtered_df = filtered_df.fillna('')
    c_df = filtered_df[filtered_df['catastropic'].str.lower().str.contains('yes')]
    cp_df = c_df[c_df['caught after release? '].str.lower().str.contains('yes') | c_df['caught after release? '].str.lower().str.contains("but it's found ")]
    cs.append(len(filtered_df))
    ccs.append(len(c_df))
    cps.append(len(cp_df))

table2_keys = ['Whole cluster down (all nodes crash, master node crash', 
               'Severe service quality degradation during rolling upgrade',
               'Data loss and data corruption',
               'Performance degradation (increased latency, wasted computation, etc.)',
               'Part of cluster down (part of worker nodes down, secondary master down)',
               'Incorrect service result (failed read/write requests, UI error, etc.)',
               'Unknown',
              ]

table2 = pd.DataFrame([table2_keys, cs,ccs,cps]).transpose()
table2.columns = ['Symptom','All', 'Catastrophic', 'Catastrophic in Production']
table2

,Symptom,All,Catastrophic,Catastrophic in Production
0,"Whole cluster down (all nodes crash, master no...",34,34,18
1,Severe service quality degradation during roll...,16,16,10
2,Data loss and data corruption,20,15,12
3,"Performance degradation (increased latency, wa...",10,4,4
4,Part of cluster down (part of worker nodes dow...,12,7,3
5,Incorrect service result (failed read/write re...,24,6,4
6,Unknown,7,1,0
